# Requirements for this program to run.
* Install the following via Anaconda Prompt
    - pyqt5 (pip install pyqt5)
    - pyqt5-tools (pip install pyqt5-tools)
    - tensorflow
    - imageai
* Trianed Model and .json file (Both must be located as the same path of this file)
* Qt Designer
* Pandas

In [1]:
# * Install the following via Anaconda Prompt
#-------------------------------------------------
# pip install opencv-python==4.1.2.30
# pip uninstall keras-nightly -y
# pip install keras==2.3.1
# pip install tensorflow==1.14.0
# pip install 'h5py==2.10.0' --force-reinstall
# pip install imageai==2.1.5
# pip install pyqt5
# conda install -c anaconda pandas

## Alzheimer's Disease Stage Classification System Code

In [1]:
#Importing Libraries
#-------------------------------------------------
from PyQt5 import QtWidgets, QtGui, QtCore
from PyQt5.QtWidgets import QApplication, QMainWindow, QDesktopWidget, QFileDialog, QMessageBox
from imageai.Detection.Custom import CustomObjectDetection
from imageai.Detection.Custom import CustomVideoObjectDetection
import os
import cv2
import sys
import csv 
import time
import csv
import argparse 
import datetime 
import pandas as pd
#-------------------------------------------------

#Application's Details
#-------------------------------------------------
class MyWindow(QMainWindow):
    def __init__(self):
        super(MyWindow, self).__init__()
        #Set Fixed Resolution Size to 600
        self.setFixedSize(800, 600)
        self.setWindowTitle("ALZHEIMER'S DISEASE STAGE CLASSIFICATION GUI")
        #Background Color
        self.setStyleSheet("background-color: rgb(rgb(200, 198, 200))")
        self.initUI()
        self.center()
#-------------------------------------------------
        
#Set Form to Center Screen
#-------------------------------------------------
    def center(self):
        qr = self.frameGeometry()
        cp = QDesktopWidget().availableGeometry().center()
        qr.moveCenter(cp)
        self.move(qr.topLeft())
#-------------------------------------------------
    
#Code for widgets (Labels and Buttons)
#-------------------------------------------------
    def initUI(self):
            #Icon
            icon = QtGui.QIcon()
            icon.addPixmap(QtGui.QPixmap("art/object_detection_logo2.png"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
            self.setWindowIcon(icon)
            self.setWindowOpacity(30.0)
            self.setToolButtonStyle(QtCore.Qt.ToolButtonTextBesideIcon)
            self.setDocumentMode(False)
            
            #Title Label
            self.title = QtWidgets.QLabel(self)
            self.title.setText("ALZHEIMER'S DISEASE STAGE CLASSIFICATION USING DEEP LEARNING")    
            self.title.setFont(QtGui.QFont('Berlin Sans FB', 12))
            self.title.setStyleSheet("font-weight: bold");
            self.title.setAlignment(QtCore.Qt.AlignCenter)
            self.title.setGeometry(QtCore.QRect(0, 40, 801, 61))

            #Label 2
            self.label2 = QtWidgets.QLabel(self)
            self.label2.setText("Detects 3 Stages of Alzheimer's Disease")    
            self.label2.setFont(QtGui.QFont('Berlin Sans FB', 12))
            self.label2.setAlignment(QtCore.Qt.AlignCenter)
            self.label2.setGeometry(QtCore.QRect(0, 90, 791, 31))
            
            #Label 3
            self.label3 = QtWidgets.QLabel(self)
            self.label3.setText("Non Demented • Mild Demented • Very Mild Demented")    
            self.label3.setFont(QtGui.QFont('Berlin Sans FB', 12))
            self.label3.setAlignment(QtCore.Qt.AlignCenter)
            self.label3.setGeometry(QtCore.QRect(0, 120, 791, 31))
                 
            #object detection Icon
            self.iconlabel = QtWidgets.QLabel(self)
            self.iconlabel.setText("")
            self.iconlabel.setPixmap(QtGui.QPixmap("art/brain.png"))
            self.iconlabel.setScaledContents(True)
            self.iconlabel.setGeometry(QtCore.QRect(700, 20, 80, 80))
            
            #yolov3 Label
            self.dept = QtWidgets.QLabel(self)
            self.dept.setText("TRAINED USING YOLO V3 ALGORITHM")    
            self.dept.setFont(QtGui.QFont('Berlin Sans FB', 9))
            self.dept.setScaledContents(True)
            self.dept.setWordWrap(False)
            self.dept.setGeometry(QtCore.QRect(300, 560, 801, 31))
            
            #Image Label
            self.Imglabel = QtWidgets.QLabel(self)
            self.Imglabel.setText("IMAGE")    
            self.Imglabel.setFont(QtGui.QFont('Berlin Sans FB', 16))
            self.Imglabel.setAlignment(QtCore.Qt.AlignCenter)
            self.Imglabel.setGeometry(QtCore.QRect(90, 190, 111, 41))

            
            #Image Button
            self.ImgBtn = QtWidgets.QPushButton(self)
            self.ImgBtn.setGeometry(QtCore.QRect(60, 230, 165, 165))
            self.ImgBtn.setText("")
            self.ImgBtn.setToolTip('Import image')
            
            
            #Image Icon
            imgbtnicon = QtGui.QIcon()
            imgbtnicon.addPixmap(QtGui.QPixmap("art/image_icon.ico"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
            self.ImgBtn.setIcon(imgbtnicon)
            self.ImgBtn.setIconSize(QtCore.QSize(160, 160))
            self.ImgBtn.setFlat(True)
            
            #Image Button Click Event
            self.ImgBtn.clicked.connect(self.getImage)
            
            #Video Label
            self.Vidlabel = QtWidgets.QLabel(self)
            self.Vidlabel.setText("VIDEO")
            self.Vidlabel.setFont(QtGui.QFont('Berlin Sans FB', 16))
            self.Vidlabel.setAlignment(QtCore.Qt.AlignCenter)
            self.Vidlabel.setGeometry(QtCore.QRect(590, 180, 101, 51))
            
            #Video Button
            self.VidBtn = QtWidgets.QPushButton(self)
            self.VidBtn.setGeometry(QtCore.QRect(560, 230, 165, 165))
            self.VidBtn.setText("")
            self.VidBtn.setToolTip('Import video') 
            
            #Video Icon
            vidbtnicon = QtGui.QIcon()
            vidbtnicon.addPixmap(QtGui.QPixmap("art/video_icon.ico"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
            self.VidBtn.setIcon(vidbtnicon)
            self.VidBtn.setIconSize(QtCore.QSize(160, 160))
            self.VidBtn.setFlat(True)                        
            
            #Button Click Event
            self.VidBtn.clicked.connect(self.getVideo)
           
            #Live Label
            self.livelabel = QtWidgets.QLabel(self)
            self.livelabel.setText("WEBCAM")
            self.livelabel.setFont(QtGui.QFont('Berlin Sans FB', 16))
            self.livelabel.setAlignment(QtCore.Qt.AlignCenter)
            self.livelabel.setGeometry(QtCore.QRect(315, 180, 171, 51))
            
            #Live Button
            self.livebtn = QtWidgets.QPushButton(self)
            self.livebtn.setGeometry(QtCore.QRect(320, 230, 165, 165))
            self.livebtn.setText("")
            self.livebtn.setToolTip('Use webcam')
            
            #Live Icon
            livebtnicon = QtGui.QIcon()
            livebtnicon.addPixmap(QtGui.QPixmap("art/webcam_icon.ico"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
            self.livebtn.setIcon(livebtnicon)
            self.livebtn.setIconSize(QtCore.QSize(160, 160))
            self.livebtn.setFlat(True) 
            
            #Button Click Event
            self.livebtn.clicked.connect(self.Livestream)
            
#-------------------------------------------------

#Select Image from your Computer to Detect
#-------------------------------------------------
    def getImage(self):
        fname = QFileDialog.getOpenFileName(self, 'Open file', 'c:/',"Image files (*.jpg *.png)")
        #Setting Variable name "imagePath" to global so it can be called at def DetectImage()
        global imagePath
        imagePath = fname[0]
        self.DetectImage()
        
#Code For Single Image Detection
#-------------------------------------------------
    def DetectImage(self):
        from datetime import datetime
        today = datetime.now()
        
       
        #Main Code for Hololens
        detector = CustomObjectDetection()
        detector.setModelTypeAsYOLOv3()
        #Model Used
        detector.setModelPath("detection_model-ex-024--loss-0000.169.h5")
        detector.setJsonPath("detection_config.json")
        detector.loadModel()
        detections = detector.detectObjectsFromImage(input_image=imagePath, 
                                                     output_image_path= imagePath + "_detected_alzheimer_stage.jpg", 
                                                     minimum_percentage_probability=30)

        #Create CSV File from the image
        with open('single-image.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            #columns name
            writer.writerow(['CLASS', 'PERCENTAGE'])
            for (eachObject) in detections:
                writer.writerow([eachObject["name"], eachObject["percentage_probability"]])

        self.popupImage()
        
#Show popup message after finishing the Image Detection
#-------------------------------------------------
    def popupImage(self):
        msg = QMessageBox()
        msg.setWindowTitle("Single-Image Detection")
        msg.setText("Single Detection Finish")
        msg.setIcon(QMessageBox.Information)
        x = msg.exec_()


#Select Video from your Computer to Detect
#-------------------------------------------------
    def getVideo(self):
        vidname = QFileDialog.getOpenFileName(self, 'Open file', 'c:/',"Video files (*.avi *.mp4)")
        #Setting Variable name "aviPath" to global so it can be called at def DetectVideo()
        global aviPath
        aviPath = vidname[0]
        #call def DectectVideo
        self.DetectVideo()
        
#Code for Video Detection
#-------------------------------------------------
    def DetectVideo(self):
        from datetime import datetime
        classid = []
        percentage =[]
        execution_path = os.getcwd()

    # FOR DETECTION
        def forFrame(frame_number, output_array, output_count):
            for eachObject in output_array:
                classid.append(eachObject["name"])
                percentage.append(eachObject["percentage_probability"])
                
        rec_detector = CustomVideoObjectDetection()
        rec_detector.setModelTypeAsYOLOv3()
        rec_detector.setModelPath("detection_model-ex-024--loss-0000.169.h5")
        rec_detector.setJsonPath("detection_config.json")
        rec_detector.loadModel()
        rec_detector.detectObjectsFromVideo(input_file_path=aviPath,
                                                  output_file_path=os.path.join(execution_path, aviPath + "detected_hololens"),
                                                  per_frame_function=forFrame,
                                                  minimum_percentage_probability=30,
                                                  log_progress=True)
        
        
        #Create CSV File from the image
        with open('video-image.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            #columns name
            writer.writerow(['CLASS', 'PERCENTAGE'])
            for (classid_x, percentage_x) in zip (classid, percentage):
                writer.writerow([classid_x, percentage_x])
        self.popupVid()
        
        
#Show popup message after finishing the Video Detection
#-------------------------------------------------
    def popupVid(self):
        msg2 = QMessageBox()
        msg2.setWindowTitle("Video Detection")
        msg2.setText("Video Dectection Finish")
        msg2.setIcon(QMessageBox.Information)
        y = msg2.exec_()        
            

#Code for Live Detection
#-------------------------------------------------
    def Livestream(self):
        from datetime import datetime
        classid = []
        percentage =[]
        execution_path = os.getcwd() 
        cap = cv2.VideoCapture(0)

    # FOR DETECTION
        #Get Values
        def forFrameLive(frame_number, output_array, output_count):
            for eachObject in output_array:
                classid.append(eachObject["name"])
                percentage.append(eachObject["percentage_probability"])
                
        video_detector = CustomVideoObjectDetection()
        video_detector.setModelTypeAsYOLOv3()
        #Model Used
        video_detector.setModelPath("detection_model-ex-024--loss-0000.169.h5")
        video_detector.setJsonPath("detection_config.json")
        video_detector.loadModel()
        video_detector.detectObjectsFromVideo(camera_input=cap, 
                                                            output_file_path=os.path.join(execution_path, "LiveFeedOutput"),
                                                            per_frame_function=forFrameLive,  minimum_percentage_probability=50,
                                                            detection_timeout=6)

        
        #Create CSV File from the video
        with open('live_data.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            #columns name
            writer.writerow(['CLASS', 'PERCENTAGE'])
            for (classid_x, percentage_x) in zip (classid, percentage):
                writer.writerow([classid_x, percentage_x])
        self.popupLive()
        
#Show popup Window after Finishing the Live Detection
#-------------------------------------------------
    def popupLive(self):
        msg3 = QMessageBox()
        msg3.setWindowTitle("Live Detection")
        msg3.setText("Live Dectection Finish")
        msg3.setIcon(QMessageBox.Information)
        z = msg3.exec_()
        
#Open GUI
if __name__ == "__main__":
    app = QApplication(sys.argv)
    win = MyWindow()
    win.show()
    sys.exit(app.exec_())

Using TensorFlow backend.
C:\Users\RYZEN\.conda\envs\myenv2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\RYZEN\.conda\envs\myenv2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\RYZEN\.conda\envs\myenv2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\RYZEN\.conda\envs\myenv2\lib\site-packages\t

SystemExit: 0

C:\Users\RYZEN\.conda\envs\myenv2\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#Select webcam  from your Computer to Detect
#-------------------------------------------------
#    def getLive(self):
#        camera = QFileDialog.getOpenFileName(self, 'Open file', 'c:/',"Video files (*.avi *.mp4)")
#        #Setting Variable name "livePath" to global so it can be called at def Livestream()
#        global livePath
#        livePath = camera[0]
#        #call def Livestream
#        self.Livestream()
        